# [실습] 벡터 데이터베이스 기반 RAG 어플리케이션 만들기

RAG는 Retrieval-Augmented Generation (RAG) 의 약자로,   
LLM의 작동 과정에 검색을 결합하여 답변 성능을 높이는 어플리케이션입니다.   


이번 실습에서는 뉴스 검색 데이터를 이용한 RAG를 수행해 보겠습니다.    
코랩 GPU 사용을 위한 설정이 필요합니다.

### <필수> 실습을 진행하기 전, GPU를 T4로 설정해 주세요!

## 라이브러리 설치  

랭체인 관련 라이브러리와 벡터 데이터베이스 라이브러리를 설치합니다.  
<br>

`sentence_transformers`: 트랜스포머 계열의 공개 임베딩 모델을 사용할 수 있습니다.    
`langchain_chroma`: ChromaDB를 이용해 벡터 데이터베이스를 구성합니다.

In [ ]:
!pip install dotenv langchain_huggingface sentence_transformers jsonlines langchain langchain-openai langchain-community langchain_chroma

In [ ]:
# numpy 2.2.5~2.2.6 버전 문제로 numpy 다운그레이드 필요
!pip install "numpy<2"

**코랩에서 실행 시, Restart 메시지가 나타납니다. 런타임을 재시작하여 패키지를 정리합니다.**

## LLM과 임베딩 모델 구성하기   

이번 실습에서는 LLM 모델과 함께 임베딩 모델이 필요합니다.   
임베딩 모델은 텍스트를 벡터로 변환하며,    
이후 결과를 벡터 DB에 저장해 검색할 수 있습니다.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv('env', override=True)

if os.environ.get('OPENAI_API_KEY'):
    print('OpenAI API 키 확인')

OpenAI API 키 확인


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini", temperature = 0.7, max_tokens = 4096)

OpenAI의 `text-embedding-3-large` 는 빠른 속도로 연산이 가능하나, 비용이 발생하며 온라인 모델입니다.   
이에 따라, 폐쇄망/온프레미스 환경에서는 공개 임베딩 모델을 사용하여 구현해야 합니다.

In [ ]:
from langchain_openai import OpenAIEmbeddings
openai_embeddings = OpenAIEmbeddings(model='text-embedding-3-large')
# 2024.1 출시 (낮은 성능)

허깅페이스에 게시된 공개 모델을 불러옵니다.   
오픈 임베딩 모델에서 중요한 파라미터는 다음과 같습니다.

- 파라미터 수 : 큰 임베딩 모델의 크기는 LLM에 육박합니다. GPU를 고려하여 선택합니다.
- Max Tokens: 임베딩 모델의 최대 토큰보다 큰 데이터를 입력하면, 앞부분만을 이용해 계산하게 되므로 적절한 검색이 되지 않을 수 있습니다.
- 임베딩 차원: 큰 차원의 벡터를 생성하는 임베딩 모델은 검색 속도가 감소합니다.

현재 한국어 데이터를 임베딩하기 위해 자주 사용하는 모델은 아래와 같습니다.

- BGE-M3 (2GB, 8194 토큰 제한)   
BGE-M3 시리즈는 BAAI의 임베딩 모델로, 현재 가장 인기가 많은 모델입니다.

- KURE-V1 (2GB, 8194 토큰 제한)    
KURE 임베딩은 고려대학교 NLP 연구실에서 만든 모델로, BGE-M3를 한국어 텍스트로 파인 튜닝한 모델입니다.

- Qwen-3 Embedding (LLM 기반, 0.6, 4, 8B)     
Qwen 3 Embedding은 2025년 5월 출시된  Qwen 3 LLM을 기반으로 만들어진 임베딩 모델로, 현재 다국어 임베딩 벤치마크에서 2,3,4등을 달성한 모델입니다.

Qwen 3 임베딩 모델을 불러옵니다.

In [ ]:
!rm -rf ~/.cache/huggingface


In [ ]:
from sentence_transformers import SentenceTransformer
import torch

# HuggingFace 임베딩 주소 지정하기
# intfloat/multilingual-e5-small , baai/bge-m3, 등의 주소를 입력하여 지정
# GPU에 여유가 있다면 Qwen3 Embedding의 큰 사이즈 (4B, 8B)?

model_name = 'Qwen/Qwen3-Embedding-0.6B'
#실제 주소: https://huggingface.co/Qwen/Qwen3-Embedding-0.6B

# CPU 설정으로 모델 불러오기
emb_model = SentenceTransformer(model_name, device='cpu',model_kwargs={'torch_dtype':torch.bfloat16})

# 로컬 폴더에 모델 저장하기
emb_model.save('./embedding')

# 모델 메모리에서 삭제
del emb_model
import gc
gc.collect()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

OSError: Can't load the model for 'Qwen/Qwen3-Embedding-0.6B'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'Qwen/Qwen3-Embedding-0.6B' is the correct path to a directory containing a file named pytorch_model.bin, tf_model.h5, model.ckpt or flax_model.msgpack.

파일 시스템에 저장한 오픈 모델은 HuggingFaceEmbeddings로 불러옵니다.

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# 허깅페이스 포맷의 임베딩 모델 불러오기
open_embeddings = HuggingFaceEmbeddings(model_name= './embedding',
                                   model_kwargs={'device':'cuda'}) # gpu 사용하기

# GPU 로드된 것 확인

HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: './embedding'.

RAG를 하기 전, 비교를 위해 LLM에게 질문해 보겠습니다.

In [ ]:
# Test
llm.invoke("도메인 특화 언어 모델이란 무엇입니까? 어떤 예시가 있나요?")

## 데이터 준비하기    
네이버 API를 통해, 검색어에 대한 뉴스 기사 링크를 가져오겠습니다.    


In [ ]:
import requests
def get_naver_news_links(query, num_links=100):
    """
    query와 num_links를 입력받아 네이버 검색 수행, 네이버 뉴스 URL의 기사만 수집
    """

    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display={num_links}&sort=sim"
    # 최대 100개의 결과를 표시
    headers = {
        'X-Naver-Client-Id': 'Ko6yIqbV2TOHq9rPH8tu',
        'X-Naver-Client-Secret': 'BvqX8mNtHu'
    }

    response = requests.get(url, headers=headers)
    result = response.json()
    # 특정 링크 형식만 필터링
    filtered_links = []
    for item in result['items']:
        link = item['link']
        if "n.news.naver.com/mnews/article/" in link:
            # 네이버 뉴스 스타일만 모으기
            filtered_links.append(link)

    # 결과 출력
    print(query, ':', len(filtered_links), 'Example:', filtered_links[0])
    # for link in filtered_links:
    #     print(link)

    return filtered_links

filtered_links = []
for topic in ['도메인 특화 언어모델', 'OpenAI', 'GPT', '구글', '가전제품', '넷플릭스']:
    filtered_links += get_naver_news_links(topic, 100)
print('Total Articles:', len(filtered_links))
print('Total Articles(Without Duplicate):',len(list(set(filtered_links))))
filtered_links = list(set(filtered_links))

도메인 특화 언어모델 : 24 Example: https://n.news.naver.com/mnews/article/031/0000972728?sid=105
OpenAI : 28 Example: https://n.news.naver.com/mnews/article/123/0002369759?sid=101
GPT : 90 Example: https://n.news.naver.com/mnews/article/021/0002743835?sid=102
구글 : 89 Example: https://n.news.naver.com/mnews/article/008/0005264957?sid=105
가전제품 : 28 Example: https://n.news.naver.com/mnews/article/082/0001349630?sid=101
넷플릭스 : 36 Example: https://n.news.naver.com/mnews/article/003/0013544425?sid=104
Total Articles: 295
Total Articles(Without Duplicate): 295


## LangChain Document Loaders

LangChain의 `document_loaders`는 다양한 형식의 파일을 불러올 수 있습니다.   
[https://python.langchain.com/docs/integrations/document_loaders/ ]    

Web URL로부터 페이지를 로드하는 기본 파서인 `WebBaseLoader`를 사용합니다.   

In [ ]:
# # Jupyter 분산 처리를 위한 설정 (코랩에서는 불필요)
# import nest_asyncio

# nest_asyncio.apply()

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

async def get_news_documents(links):
    loader = WebBaseLoader(
        web_paths=links,
        bs_kwargs={'parse_only':bs4.SoupStrainer(class_=("newsct", "newsct-body"))},
                                # newsct, newsct-body만 추출 : 네이버 뉴스 포맷 HTML 요소

        requests_per_second = 10, # 1초에 10개 요청 보내기
        show_progress = True # 진행 상황 출력
    )
    # docs = loader.load() # 기본 코드

    docs = []

    async for doc in loader.alazy_load(): # 순차적 로드 대신 비동기 처리
        docs.append(doc)
    return docs

docs = await get_news_documents(filtered_links)

Fetching pages: 100%|##########| 295/295 [00:08<00:00, 36.15it/s]


In [ ]:
docs[12]

Document(metadata={'source': 'https://n.news.naver.com/mnews/article/655/0000027792?sid=101'}, page_content='\n\n\n\n\nCJB청주방송\n\nCJB청주방송\n\n\n구독\n\nCJB청주방송 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.\n보러가기\n\n\nCJB청주방송 언론사 구독 해지되었습니다.\n\n\n\n\nPICK\n안내\n\n\n언론사가 주요기사로선정한 기사입니다.\n언론사별 바로가기\n닫기\n\n\n\n\n삼성전자·SK하이닉스 강세...반도체 장비주 ‘급등세’\n\n\n\n\n\n\n\n\n\n장원석 기자\n\n\n\n\n\n\n\n\n장원석 기자\n\n\n\n\n장원석 기자\n\n구독\n구독중\n\n\n\n\n구독자\n0\n\n\n응원수\n0\n\n\n\n더보기\n\n\n\n\n\n\n\n\n\n입력\n2025.10.10. 오후 2:24\n\n\n\n기사원문\n \n\n\n\n\n\n\n\n\n\n\n추천\n\n\n\n\n쏠쏠정보\n0\n\n\n\n\n흥미진진\n0\n\n\n\n\n공감백배\n0\n\n\n\n\n분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇\n\n\n\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.\n닫기\n\n\n\n\n\n\n\n\n텍스트 음성 변환 서비스 사용하기\n\n\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름\n\n이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.\n본문듣기 시작\n\n닫기\n\n\n \n\n글자 크기 변경하기\n\n\n\n가1단계\n작게\n\n\n가2단계\n보통\n\n\n가3단계\n크게\n\n\n가4단계\n아주크게\n\n\n가5단계\n최대크게\

크롤링 결과에는 불필요한 문자가 많이 포함되어 있습니다.    
전처리를 통해 이를 제거합니다.

In [ ]:
import re

def preprocess(docs):
    noise_texts = [
        '''구독중 구독자 0 응원수 0 더보기''',
        '''쏠쏠정보 0 흥미진진 0 공감백배 0 분석탁월 0 후속강추 0''',
        '''댓글 본문 요약봇 본문 요약봇''',
        '''도움말 자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다. 닫기''',
        '''텍스트 음성 변환 서비스 사용하기 성별 남성 여성 말하기 속도 느림 보통 빠름''',
        '''이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다. 본문듣기 시작''',
        '''닫기 글자 크기 변경하기 가1단계 작게 가2단계 보통 가3단계 크게 가4단계 아주크게 가5단계 최대크게 SNS 보내기 인쇄하기''',
        'PICK 안내 언론사가 주요기사로선정한 기사입니다. 언론사별 바로가기 닫기',
        '응원 닫기',
        '구독 구독중 구독자 0 응원수 0 ',
    ]

    def clean_text(doc):
        text = doc.page_content
        # 탭과 개행문자를 공백으로 변환
        text = text.replace('\t', ' ').replace('\n', ' ')

        # 연속된 공백을 하나로 치환
        text = re.sub(r'\s+', ' ', text).strip()

        # 여러 구분자를 한번에 처리
        split_markers = [
            '구독 해지되었습니다.',
            '구독 메인에서 바로 보는 언론사 편집 뉴스 지금 바로 구독해보세요!'
        ]
        for marker in split_markers:
            parts = text.split(marker)
            if len(parts) > 1:
                if marker == '구독 해지되었습니다.':
                    text = parts[1]  # 뒷부분 사용
                else:
                    text = parts[0]  # 앞부분 사용

        # 노이즈 텍스트 제거
        for noise in noise_texts:
            text = text.replace(noise, '')

        # 연속된 공백을 하나로 치환
        text = re.sub(r'\s+', ' ', text).strip()
        doc.page_content = text
        return doc

    preprocessed_docs = []
    for doc in docs:

        # 텍스트 정제
        doc= clean_text(doc)
        preprocessed_docs.append(doc)

    return preprocessed_docs

preprocessed_docs = preprocess(docs)


In [ ]:
preprocessed_docs[2]

Document(metadata={'source': 'https://n.news.naver.com/mnews/article/003/0013534408?sid=104'}, page_content='구글 "印에 5년간 21조5280억원 투자…최대 규모 AI 허브 건설" 유세진 기자 유세진 기자 유세진 기자 구독 입력 2025.10.14. 오후 8:35 수정 2025.10.14. 오후 10:17 기사원문 추천 320만㎞ 이상 기존 육상·해저 케이블과 연결 새 국제 해저 게이트웨이 건설"커다 경제·사회적 기회 창출과 동시에 AI 능력의 세대교체 이끌 것" [마운틴뷰(미 캘리포니아주)=AP/뉴시스]나렌드라 모디 인도 총리(오른쪽)가 2015년 9월27일 미 캘리포니아주 마운틴뷰의 구글 본사에서 순다르 피차이 구글 최고경영자(CEO)로부터 설명을 듣고 있다. 구글이 인도에 향후 5년 간 150억 달러(21조5280억원)를 투자해 인도 최초의 인공지능(AI) 허브를 구축하겠다고 14일 밝혔다. 2025.10.14.[뉴델리(인도)=AP/뉴시스] 유세진 기자 = 구글이 인도에 향후 5년 간 150억 달러(21조5280억원)를 투자해 인도 최초의 인공지능(AI) 허브를 구축하겠다고 14일 밝혔다.인도 남부 도시 비사카파트남에 위치할 이 AI 허브는 구글 역사상 세계 최대 규모의 허브 중 하나가 될 것이다. 구글은 기가와트 규모의 데이터 센터 운영, 광범위한 에너지 인프라, 확장된 광섬유 네트워크를 특징으로 할 것이라고 밝혔다.이번 투자는 구글이 AI 지배를 위한 글로벌 경쟁에서 핵심 기술과 인재 기반으로서 인도에 대한 의존도가 커지고 있음을 강조한다. 인도로서는 디지털 전환 야망을 가속화할 수 있는 규모로 고부가가치 인프라와 외국인 투자를 유치하게 된다.구글은 자사의 AI 허브 투자에 320만㎞ 이상의 기존 육상 및 해저 케이블과 연결되는 새로운 국제 해저 게이트웨이 건설이 포함될 것이라고 말했다."이 이니셔티브는 인도와 미국 모두에게 상당한 경제적, 사회적 기회를 창출하는 동

불러온 텍스트 데이터는 파일로 저장할 수 있습니다.

In [ ]:
# 불러온 document 저장하기

import jsonlines
def save_docs_to_jsonl(documents, file_path):
    with jsonlines.open(file_path, mode="w") as writer:
        for doc in documents:
            writer.write(doc.model_dump())

# jsonl 파일 불러오기
from langchain_core.documents import Document

def load_docs_from_jsonl(file_path):
    documents = []
    with jsonlines.open(file_path, mode="r") as reader:
        for doc in reader:
            documents.append(Document(**doc))
    return documents

In [ ]:
# 저장
save_docs_to_jsonl(preprocessed_docs, "docs.jsonl")

## Chunking: 청크 단위로 나누기   



전처리가 완료된 docs를 chunk 단위로 분리합니다.
`chunk_size`와 `chunk_overlap`을 이용해 청크의 구성 방식을 조절할 수 있습니다.

Chunk Size * K(검색할 청크의 수) 의 결과가 Context의 길이가 됩니다.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain import hub

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# 0~1000, 800~1800, 1600~2600, ...
# 실제로는 구분자(공백,엔터,탭 등) 를 기준으로 자르므로 정확히 일치하지는 않음
chunks = text_splitter.split_documents(preprocessed_docs)
print(len(chunks))

991


## Vector DB 구성하기

구성된 청크를 ChromaDB 벡터 데이터베이스에 로드합니다.   

In [ ]:
from langchain_chroma import Chroma

Chroma().delete_collection() # (메모리에 저장하는 경우) 기존 데이터 삭제

# DB 구성하기
db = Chroma(embedding_function=openai_embeddings,
            persist_directory="./chroma_OpenAI",
            # 파일 시스템에 저장 (생략시 메모리에 저장)

            collection_name='Web', # 식별 이름

            collection_metadata={'hnsw:space':'l2'},
            # l2 메트릭 설정(기본값, cosine, mmr 로 변경 가능)
            )

DB에 document를 추가합니다.    
OpenAI 임베딩은 30만 토큰 동시 처리 제한이 있어, 나눠서 전달합니다.

In [ ]:
from tqdm import tqdm

print(len(chunks))
# 300,000 토큰 제한

# 100개씩 추가
for i in tqdm(range(0, len(chunks), 100)):
    db.add_documents(chunks[i:min(i+100, len(chunks))])

991


100%|██████████| 10/10 [00:24<00:00,  2.43s/it]


db로부터 retriever를 구성합니다.

In [ ]:
# Top 5 Search(기본값은 4)
retriever = db.as_retriever(search_kwargs={'k':5})

# Top 5 Chuck * Chuck Size 1000 = 5000 글자 Context

In [ ]:
retriever.invoke("도메인 특화 언어 모델")

[Document(id='01e6e678-c792-40c1-8e64-5fe93d3e7715', metadata={'source': 'https://n.news.naver.com/mnews/article/015/0005163430?sid=105'}, page_content='SKT·크래프톤, 언어모델 공동 개발…"수학 추론에 특화" 박수빈 기자 박수빈 기자 박수빈 기자 구독 입력 2025.07.28. 오전 10:07 기사원문 추천 게임 AI 고도화 위한 한국형 LLM도메인 특화 AI 모델 개발 역량 입증 SK텔레콤 사옥과 크래프톤 로고. 사진=SK텔레콤, 한경DBSK텔레콤과 크래프톤이 공동으로 개발한 언어모델을 공개했다. 크래프톤은 이 모델에 적용한 학습 기법을 게임 특화형 AI 응용 기술을 고도화하는 데 활용한다. SK텔레콤은 28일 크래프톤과 공동으로 개발한 7B(70억개 파라미터) 규모의 추론 특화 언어 모델 3종을 공개했다.이번에 공개한 모델은 수학 문제 해결과 코드 개발에 특화된 소형 언어 모델이다. 크래프톤이 독자적으로 개발한 학습 기법을 적용한 것이 특징이다.이 기법을 바탕으로 해당 모델은 수학 추론 벤치마크 AIME 25에서 뚜렷한 성능 향상을 기록했다. 수학은 게임을 포함한 고난도 추론 분야와 기술적으로 밀접한 연관성을 갖는다. 공간지각과 논리 추론 역량이 요구돼서다. 크래프톤은 해당 모델을 기반으로 게임 중심의 AI 기술 확장 가능성도 기대하고 있다. 양사 간 협력은 도메인 특화 AI 모델 개발 역량을 입증한 사례로 평가된다. SK텔레콤과 크래프톤은 각각 인프라를 구축하고 학습 기법을 개선해 모델 품질과 성능을 고도화했다.크래프톤은 기존 모델의 취약점을 분석해 이를 개선하는 오답 복기 학습 기법을 자체 개발했다. 틀린 문제의 정답을 찾아 오답과 비교해 학습하며 추론 정확도와 효율성을 동시에 확보하는 전략적 학습 방식이다. SK텔레콤은 데이터 검증과 모델 학습의 인프라 구축을 담당해 모델의 품질과 안정성을 확보했다.크래프톤은 이번 언어 모델에 적

## Prompting

RAG를 위한 간단한 프롬프트를 작성합니다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate([
    ("user", '''당신은 QA(Question-Answering)을 수행하는 Assistant입니다.
다음의 Context를 이용하여 Question에 답변하세요.
정확한 답변을 제공하세요.
만약 모든 Context를 다 확인해도 정보가 없다면,
"정보가 부족하여 답변할 수 없습니다."를 출력하세요.
---
Context: {context}
---
Question: {question}''')])

prompt.pretty_print()

================================ Human Message =================================

당신은 QA(Question-Answering)을 수행하는 Assistant입니다.
다음의 Context를 이용하여 Question에 답변하세요.
정확한 답변을 제공하세요.
만약 모든 Context를 다 확인해도 정보가 없다면,
"정보가 부족하여 답변할 수 없습니다."를 출력하세요.
---
Context: {context}
---
Question: {question}


## Chain

RAG를 수행하기 위한 Chain을 만듭니다.

RAG Chain은 프롬프트에 context와 question을 전달해야 합니다.    
Question을 입력받아, Context를 함께 프롬프트에 전달합니다.

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# retriever의 결과물은 List[Document] 이므로 이를 ---로 구분하는 함수
# metadata의 source를 보존하여 추가
def format_docs(docs):
    return " \n---\n ".join(['URL: '+ doc.metadata['source'] + '\nContent: '+ doc.page_content+ '\n' for doc in docs])
    # join : 구분자를 기준으로 스트링 리스트를 하나의 스트링으로 연결

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    # retriever : question을 받아서 context 검색: document 반환
    # format_docs : document 형태를 받아서 텍스트로 변환
    # RunnablePassthrough(): 체인의 입력을 그대로 저장
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("도메인 특화 언어 모델이란 무엇입니까? 어떤 예시가 있나요?")

'도메인 특화 언어 모델이란 특정 산업 분야나 주제 영역(도메인)에 최적화되어 개발된 언어 모델을 말합니다. 이러한 모델은 해당 도메인에서 요구되는 전문 지식, 용어, 문제 해결 방식 등에 맞춰 학습되어, 일반 범용 모델보다 높은 정확도와 효율성을 보입니다.\n\n예시로는 다음과 같은 도메인 특화 언어 모델들이 있습니다:\n\n1. **SK텔레콤과 크래프톤의 수학 추론 특화 언어 모델**  \n   - 70억 개 파라미터 규모의 소형 언어 모델로, 수학 문제 해결과 코드 개발에 특화되어 있음.  \n   - 크래프톤이 개발한 오답 복기 학습 기법을 적용해 추론 정확도와 효율성을 높였음.  \n   - 게임 AI 고도화를 위해 게임 플레이 분석 및 전략 판단 등 게임 특화 AI 응용 기술에도 활용.\n\n2. **LG유플러스와 KODATA의 기업·금융 특화 AI 파운데이션 모델**  \n   - 금융 및 기업 데이터를 바탕으로 한 맞춤형 AI 서비스 제공.  \n   - 기업 신용 평가, 재무 분석 등 금융권 업무에 특화된 대화형 AI 및 리포트 자동 생성 기능 포함.  \n   - DACP 기술을 적용해 산업별 특화 데이터로 모델 정확도와 도메인 이해도 향상.\n\n3. **코난테크놀로지의 군사 도메인 특화 LLM**  \n   - 국방 환경에 최적화된 AI 플랫폼과 맞춤형 학습을 거친 군사 분야 특화 언어 모델.  \n   - 군사정보 기반 의사결정 지원, 정보 생성 및 요약, 다국어 대응 등 기능 강화.  \n   - 최신 검색증강생성(RAG) 기술 적용.\n\n4. **포티투마루의 도메인 특화 설치형 LLM**  \n   - 전자, 통신, 금융, 자동차, 법률, 헬스케어, 국방 등 다양한 산업 분야에 맞춰 경량화 및 최적화된 언어 모델.  \n   - 질의응답, 문서 요약, 초안 작성 등 자연어 처리 기능 제공.\n\n요약하면, 도메인 특화 언어 모델은 특정 산업이나 분야의 요구에 맞추어 설계되고 학습된 AI 모델로, 해당 분야의 업무 효율성과 정확도를 

In [ ]:
rag_chain.invoke("인공지능의 최근 발전 방식은? 관련 링크도 보여주세요")

"인공지능의 최근 발전 방식은 다음과 같습니다.\n\n- AI가 사람의 목소리, 제스처, 시각을 직관적으로 이해하는 멀티모달 능력을 갖추고 있으며, 이를 기반으로 한 AI 인터페이스가 가능해지고 있습니다.\n- XR(eXtended Reality, 확장현실) 기술과 결합하여 가상현실(VR), 증강현실(AR), 혼합현실(MR) 등 현실과 가상을 융합한 새로운 경험을 제공하는 기술군이 발전하고 있습니다.\n- 구글의 제미나이(Gemini) 모델이 세일즈포스 차세대 AI 플랫폼인 '에이전트포스 360'에 탑재되어, 이메일 작성, 일정 예약, 고객관리 데이터 분석 등 다양한 업무를 처리하는 AI 에이전트로 활용되고 있습니다.\n- AI 에이전트를 안전하고 통제 가능한 방식으로 기업 환경에 통합하는 데 집중하고 있으며, 이를 통해 업무 생산성 향상과 글로벌 경쟁력 확보를 기대하고 있습니다.\n- 초지능(Super Intelligence) 단계의 AI가 AGI(Artificial General Intelligence) 등장 후 스스로 진입할 것으로 전망되며, 과학·예술·의학·공학·사회·감정 등 모든 영역에서 인간을 뛰어넘는 문제 해결 능력과 창의성, 추론 능력을 갖출 것으로 예상됩니다.\n\n관련 기사 링크:  \nhttps://n.news.naver.com/mnews/article/421/0008544124?sid=105"

In [ ]:
rag_chain.invoke("알리바바의 언어 모델 이름은?")

'정보가 부족하여 답변할 수 없습니다.'

assign()을 이용하면, 체인의 결과를 받아 새로운 체인에 전달하고, 그 결과를 가져옵니다.

In [ ]:
# assign : 결과를 받아서 새로운 인수 추가하고 원래 결과와 함께 전달

from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    context = retriever | format_docs, question = RunnablePassthrough()).assign(answer=rag_chain_from_docs)

rag_chain_with_source.invoke("인공지능의 최근 발전 방식은? 관련 링크도 보여주세요")

# retriever가 1번 실행됨
# retriever의 실행 결과를 rag_chain_from_docs 에 넘겨주기 때문에


{'context': 'URL: https://n.news.naver.com/mnews/article/421/0008544124?sid=105\nContent: 경험을 제공하는 기술군 전체를 의미한다.■ 초지능초지능(Super Intelligence)은 인간의 지능을 모든 분야에서 뛰어넘는 인공지능 시스템을 말한다. AGI 등장 시 스스로 초지능 단계로 진입할 것으로 전망한다. 과학·예술·의학·공학·사회·감정 등 모든 영역에서 인간보다 월등한 문제 해결 능력과 창의성·추론 능력을 갖출 것으로 보인다. 김민석 기자 (ideaed@news1.kr) Copyright ⓒ 뉴스1. All rights reserved. 무단 전재 및 재배포, AI학습 이용 금지. 이 기사는 언론사에서 IT, 경제 섹션으로 분류했습니다. 기사 섹션 분류 안내 기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다. 언론사는 개별 기사를 2개 이상 섹션으로 중복 분류할 수 있습니다. 닫기 기자 프로필 김민석 기자 김민석 기자 뉴스1 구독 구독중 "179만원 아이폰 이게 말이 되나"…아이폰17 변색 전세계 확산 오픈AI, 브로드컴 손잡고 \'AI 칩\' 직접설계…엔비디아 독점 균열 구독 뉴스1 구독하고 메인에서 바로 만나보세요!구독하고 메인에서 만나보세요! 구독중 뉴스1 구독하고 메인에서 바로 만나보세요!구독하고 메인에서 만나보세요! 언론사홈 뉴스1 주요뉴스해당 언론사에서 선정하며 언론사(아웃링크)로 이동합니다. 모델 꿈 안고 태국 간 미모의 20대 장기적출…동남아 전체가 \'위험지대\' "근육질 남편, 5년간 부부관계 단 1회…위자료 가능?" "결혼 전 성병 치료 아내 \'완치\' 당당, 계속 살아야 하나" \'10일간 발기\' 40대, 뒤늦게 병원갔지만 "영구 손상" "누가 오나 보려고"…장례식 열고 화장 직전 일어난 男 K팝·K트롯 팬들의 놀이터, 스타1픽 QR 코드를 클릭하면 크게 볼 수 있어요. QR을 촬영해보세요. K팝·K트롯 팬들의 놀이터, 스타1픽 닫기 세상에 이런 일이..

이번에는 오픈 모델을 사용합니다.   
오픈 임베딩을 통해 구성한 DB와 원래 DB를 비교해 보겠습니다.

In [ ]:
open_db = Chroma(embedding_function=open_embeddings,
                           persist_directory="./chroma_open", # 별도 폴더에 저장
                           collection_name='Web', # 식별 이름
                           collection_metadata={'hnsw:space':'l2'}
                           )

# 100개씩 추가
for i in tqdm(range(0, len(chunks), 100)):
    open_db.add_documents(chunks[i:min(i+100, len(chunks))])

이후는 동일합니다.

In [ ]:
open_retriever = open_db.as_retriever(search_kwargs={'k':5})

In [ ]:
open_retriever.invoke("도메인 특화 언어 모델")

In [ ]:
rag_chain_open = (
    {"context": open_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

예시 질문을 통해 정답과 검색 결과를 비교해 보겠습니다.

In [ ]:
questions = ["도메인 특화 언어 모델이란 무엇입니까? 어떤 예시가 있나요?",
             "인공지능의 최근 발전 방식은? 관련 링크도 보여주세요",
             "알리바바의 언어 모델 이름은?"]

In [ ]:
# Retriever 비교

contexts_hf = open_retriever.batch(questions)
contexts_openai = retriever.batch(questions)

In [ ]:
for i in range(len(questions)):
    print(f"-- Question:{questions[i]}")
    oai_chunks = '\n'.join([x.page_content[:30] for x in contexts_openai[i]])
    hf_chunks = '\n'.join([x.page_content[:30] for x in contexts_hf[i]])
    print(f"OpenAI: \n{oai_chunks}")
    print(f"Hf: \n{hf_chunks}")
    print('----------')



In [ ]:
# 최종 결과 비교
oai_results = rag_chain.batch(questions)
hf_results = rag_chain_open.batch(questions)

In [ ]:
for i in range(len(questions)):
    print(f"-- Question:{questions[i]}")
    print(f"OpenAI: \n{oai_results[i]}")
    print('--')
    print(f"Hf: \n{hf_results[i]}")
    print('----------')
